In [0]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [0]:
df = pd.read_csv('final_omit.csv')
df.set_index('Index', inplace=True)

In [0]:
df.head()

,Artist,Title,Mood,Lyrics
Index,,,,
ML1,Usher,There Goes My Baby,relaxed,There goes my baby (Ooh girl look at you) You ...
ML2,Da'Ville,On My Mind,relaxed,I'm thinking about you Girl I miss you Baby ev...
ML3,Rihanna,Rockstar 101,relaxed,"I told ya I told ya I told ya Baby Baby Uh, u..."
ML4,J. Holiday,Bed,relaxed,"(J. Holiday: speaking) J. Holiday L.O.S. Aye, ..."
ML5,Morgan Heritage,Don't Haffi Dread,angry,"Yeah, yeah, yeah - ragga so ragga You don't h..."


In [0]:
df.Mood.value_counts()

happy      757
relaxed    554
angry      538
sad        436
Name: Mood, dtype: int64

In [0]:
df = df[~df.Lyrics.str.contains('Not Available')]

In [0]:
df.Mood.value_counts()

happy      757
relaxed    554
angry      538
sad        436
Name: Mood, dtype: int64

In [0]:
possible_labels = df.Mood.unique()
possible_labels

array(['relaxed', 'angry', 'happy', 'sad'], dtype=object)

In [0]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict    

{'angry': 1, 'happy': 2, 'relaxed': 0, 'sad': 3}

In [0]:
df['label'] = df.Mood.replace(label_dict)

In [0]:
df.head(15)

,Artist,Title,Mood,Lyrics,label
Index,,,,,
ML1,Usher,There Goes My Baby,relaxed,There goes my baby (Ooh girl look at you) You ...,0
ML2,Da'Ville,On My Mind,relaxed,I'm thinking about you Girl I miss you Baby ev...,0
ML3,Rihanna,Rockstar 101,relaxed,"I told ya I told ya I told ya Baby Baby Uh, u...",0
ML4,J. Holiday,Bed,relaxed,"(J. Holiday: speaking) J. Holiday L.O.S. Aye, ...",0
ML5,Morgan Heritage,Don't Haffi Dread,angry,"Yeah, yeah, yeah - ragga so ragga You don't h...",1
ML6,Whitney Houston,My Love Is Your Love,happy,If tomorrow is Judgement Day And I'm standin' ...,2
ML7,Serani,Study People,angry,"Seri, Seri, Seri, Serani! Everywhere I go they...",1
ML8,Austra,Spellwork,angry,"Habit alone, when separate I moan. Feeding on ...",1
ML9,Coro,Can't Let You Go,happy,I should've known you'd be this way Serves me ...,2


In [0]:
#Training and Validation Split

from sklearn.model_selection import train_test_split

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

In [0]:
df['data_type'] = ['not_set']*df.shape[0]
df.head()

,Artist,Title,Mood,Lyrics,label,data_type
Index,,,,,,
ML1,Usher,There Goes My Baby,relaxed,There goes my baby (Ooh girl look at you) You ...,0,not_set
ML2,Da'Ville,On My Mind,relaxed,I'm thinking about you Girl I miss you Baby ev...,0,not_set
ML3,Rihanna,Rockstar 101,relaxed,"I told ya I told ya I told ya Baby Baby Uh, u...",0,not_set
ML4,J. Holiday,Bed,relaxed,"(J. Holiday: speaking) J. Holiday L.O.S. Aye, ...",0,not_set
ML5,Morgan Heritage,Don't Haffi Dread,angry,"Yeah, yeah, yeah - ragga so ragga You don't h...",1,not_set


In [0]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.head(15)

,Artist,Title,Mood,Lyrics,label,data_type
Index,,,,,,
ML1,Usher,There Goes My Baby,relaxed,There goes my baby (Ooh girl look at you) You ...,0,train
ML2,Da'Ville,On My Mind,relaxed,I'm thinking about you Girl I miss you Baby ev...,0,train
ML3,Rihanna,Rockstar 101,relaxed,"I told ya I told ya I told ya Baby Baby Uh, u...",0,val
ML4,J. Holiday,Bed,relaxed,"(J. Holiday: speaking) J. Holiday L.O.S. Aye, ...",0,train
ML5,Morgan Heritage,Don't Haffi Dread,angry,"Yeah, yeah, yeah - ragga so ragga You don't h...",1,train
ML6,Whitney Houston,My Love Is Your Love,happy,If tomorrow is Judgement Day And I'm standin' ...,2,train
ML7,Serani,Study People,angry,"Seri, Seri, Seri, Serani! Everywhere I go they...",1,train
ML8,Austra,Spellwork,angry,"Habit alone, when separate I moan. Feeding on ...",1,train
ML9,Coro,Can't Let You Go,happy,I should've known you'd be this way Serves me ...,2,train


In [0]:
df.groupby(['Mood', 'label', 'data_type']).count()

Artist  Title  Lyrics
Mood    label data_type                       
angry   1     train         403    403     403
              val           135    135     135
happy   2     train         568    568     568
              val           189    189     189
relaxed 0     train         415    415     415
              val           139    139     139
sad     3     train         327    327     327
              val           109    109     109

In [0]:
pip install transformers

In [0]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [0]:
# Tokens

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [0]:
#Encoding

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Lyrics.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=900, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Lyrics.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=900, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [0]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [0]:
len(dataset_train)

1713

In [0]:
len(dataset_val)

572

In [0]:
#Let's welcome Mr. Bert with a HuggingFace
from transformers import BertForSequenceClassification

In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)



In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [0]:
batch_size = 4 #Due to shortage of GPU

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [0]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [0]:
optimizer = AdamW(model.parameters(),
                  lr=1.6e-5, 
                  eps=1e-8)

In [0]:
epochs = 5 #Arbitary

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [0]:
import numpy as np

In [0]:
from sklearn.metrics import f1_score

In [0]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [0]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
        print(f'Accuracy (Percentage): ', (len(y_preds[y_preds==label])/len(y_true))*100, "%\n")

In [0]:
#Let's Teach/train Bert on lyrical emotions

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [0]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs) # to pass inputs dictionary
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [0]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.6169715349351887
Validation loss: 0.3293804961901445
F1 Score (Weighted): 0.8989114872856482



Epoch 2
Training loss: 0.13989908135298526
Validation loss: 0.2794361991898997
F1 Score (Weighted): 0.9372635676627394



Epoch 3
Training loss: 0.03534750657759624
Validation loss: 0.2963757662923186
F1 Score (Weighted): 0.9333088399825424



Epoch 4
Training loss: 0.01401214159174121
Validation loss: 0.3054867451424365
F1 Score (Weighted): 0.9353330139950435



Epoch 5
Training loss: 0.007999469628145088
Validation loss: 0.31303133330978716
F1 Score (Weighted): 0.9386683793247264



In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
for i in range(epoch):
  model_i=()

In [0]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [0]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [0]:
accuracy_per_class(predictions, true_vals)

Class: relaxed
Accuracy: 121/139
Accuracy (Percentage):  87.05035971223022 %

Class: angry
Accuracy: 127/135
Accuracy (Percentage):  94.07407407407408 %

Class: happy
Accuracy: 185/189
Accuracy (Percentage):  97.88359788359789 %

Class: sad
Accuracy: 103/109
Accuracy (Percentage):  94.4954128440367 %



In [0]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [0]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [0]:
accuracy_per_class(predictions, true_vals)

Class: relaxed
Accuracy: 125/139
Accuracy (Percentage):  89.92805755395683 %

Class: angry
Accuracy: 131/135
Accuracy (Percentage):  97.03703703703704 %

Class: happy
Accuracy: 184/189
Accuracy (Percentage):  97.35449735449735 %

Class: sad
Accuracy: 98/109
Accuracy (Percentage):  89.90825688073394 %



In [0]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: relaxed
Accuracy: 121/139
Accuracy (Percentage):  87.05035971223022 %

Class: angry
Accuracy: 126/135
Accuracy (Percentage):  93.33333333333333 %

Class: happy
Accuracy: 187/189
Accuracy (Percentage):  98.94179894179894 %

Class: sad
Accuracy: 101/109
Accuracy (Percentage):  92.66055045871559 %



In [0]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_4.model', map_location=torch.device('cpu')))
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: relaxed
Accuracy: 124/139
Accuracy (Percentage):  89.20863309352518 %

Class: angry
Accuracy: 131/135
Accuracy (Percentage):  97.03703703703704 %

Class: happy
Accuracy: 181/189
Accuracy (Percentage):  95.76719576719577 %

Class: sad
Accuracy: 100/109
Accuracy (Percentage):  91.74311926605505 %

